In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#计算Legendre多项式及其导数
def Legendre(x, m):
    L = np.zeros(np.max([m, 3]))
    L[0] = 1; L[1] = x; L[2] = 1 / 2 * (3 * x ** 2 - 1)
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * x * L[n] / (n + 1)- n * L[n - 1] / (n + 1)
    L = L[0 : m]
    return L
def Legendredx(x, m):
    L = np.zeros(np.max([m, 3]))
    L1 = Legendre(x, m)
    L[0] = 0; L[1] = 1; L[2] = 3 * x
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * (L1[n] + x * L[n]) / (n + 1) - n * L[n - 1] / (n + 1)
    L = L[0 : m]
    return L
def Legendred2x(x, m):
    L = np.zeros(np.max([m, 3]))
    L1 = Legendredx(x, m)
    L[0] = 0; L[1] = 0; L[2] = 3
    if m > 3:
        for n in range(2, m - 1):
            L[n + 1] = (2 * n + 1) * (2 * L1[n] + x * L[n]) / (n + 1) - n * L[n - 1] / (n + 1)
    L = L[0 : m]
    return L

In [3]:
#计算勒让德多项式及其导数
def computeLegendrefunctionandderivation(X, m):
    L = len(X)
    L1 = []
    L2 = []
    L3 = []
    for i in range(L):
        temp = X[i]
        L1.append(Legendre(temp, m))
        L2.append(Legendredx(temp, m))
        L3.append(Legendred2x(temp, m))
    L1 = np.array(L1)
    L2 = np.array(L2)
    L3 = np.array(L3)
    value_dict = {'function': L1, 'firstderivation': L2, 'secondderivation': L3}
    return value_dict

In [4]:
#计算N(x,p)的导数
def firstdx_N(w, x, dx):
    #L = len(x)
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    firstdx_N = (1 - tf.cast(N,tf.float64) ** 2) * tf.reduce_sum(tf.multiply(tf.cast(w,tf.float64), tf.cast(dx,tf.float64)))
    return firstdx_N
def seconddx_N(w, x, dx, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    seconddx_N = (1 - N ** 2) * (tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64),tf.cast(dx2,tf.float64)))-\
                                2 * N *(tf.reduce_sum(tf.multiply(tf.cast(w,tf.float64), tf.cast(dx, tf.float64))))\
                                ** 2)
    return seconddx_N

In [5]:
#计算y=x**2*N(x,p)对x的导数
def dx(w, x, x0, dx1):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    N1 = firstdx_N(w, x, dx1)
    dydx = 2 * x0 * N + N1 * (x0 ** 2)
    return dydx
def d2ydx2(w, x, x0, dx, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    N1 = firstdx_N(w, x, dx)
    N2 = seconddx_N(w, x, dx, dx2)
    d2ydx2 = 2 * N + 4 * x0 * N1 + N2 * (x0 ** 2)
    return  d2ydx2

In [6]:
#计算代价函数
def costfunction(w, x, x0 , dx1, dx2):
    z = tf.reduce_sum(tf.multiply(tf.cast(w, tf.float64), tf.cast(x, tf.float64)))
    N = tf.nn.tanh(z)
    dy = dx(w, x, x0, dx1)
    ddy2 = d2ydx2(w, x, x0, dx1, dx2)
    costfunction = ddy2 + 2 / x0 * dy + 8 * tf.exp(x0 ** 2 * N) + 4 * tf.exp(x0 ** 2 * N / 2)
    return costfunction
def loss_function(X, w, Legendrefunction, firstdx, seconddx, length):
    L = []
    for i in range(length):
        temp0 = X[i]
        temp1 = Legendrefunction[i]
        temp2 = firstdx[i,:]
        temp3 = seconddx[i,:]
        temp = costfunction(w, temp1, temp0 , temp2, temp3)
        L.append(temp)
    #L = np.array(L)
    L = tf.reduce_sum(tf.multiply(L, L)) / 2
    return L

In [7]:
x = np.linspace(0.1, 1, 500)
n = 5
m = tf.constant(n,'int8')
value_dict = computeLegendrefunctionandderivation(x, n)
Legendrefunction = value_dict['function']
firstdx = tf.constant(value_dict['firstderivation'])
seconddx = tf.constant(value_dict['secondderivation'])

In [8]:
#建立输入层
x_train = tf.placeholder('float64', [None, n], name='x_train')

In [9]:
#建立输出层
W = tf.Variable(tf.truncated_normal([n, 1],stddev = 0.1), 'float64')
y_predict = tf.nn.tanh(tf.matmul(tf.cast(x_train, tf.float64), tf.cast(W, tf.float64)))

In [10]:
#定义损失函数
loss = loss_function(X = tf.constant(x,'float64'), w = W, 
                     Legendrefunction = x_train, firstdx = firstdx, seconddx = seconddx, length=len(x))

In [11]:
#设置优化器
optimizer = tf.train.AdadeltaOptimizer(learning_rate=1).minimize(loss)

In [12]:
Epoch = 10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(Epoch):
        sess.run(optimizer, feed_dict={x_train: Legendrefunction})
        #W = tf.Variable(tf.truncated_normal([n, 1], stddev=0.1), 'float')
        #loss = loss_function(X=tf.Variable(x, 'float'), m=m, w=W)
        #optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.0001).minimize(loss)
        if (epoch % 100 == 0):
            print(sess.run(loss, feed_dict={x_train: Legendrefunction}))
    #print(sess.run(y_predict, feed_dict={x_train: Legendrefunction}))

32546.196617386027
20210.40757181504
17825.63186617535
14431.99756781394
12583.589793462257
11003.579729318742
9573.134615822011
8376.643442832617
7393.405127008737
6589.42559768958
5937.910377704051
5415.154441206985
4999.108122295507
4669.877612548577
4410.269583233841
4205.9394310537255
4045.212351587673
3918.746045973166
3819.145158160015
3740.594318575191
3678.543861039612
3629.436884749648
3590.497853928812
3559.5602993979564
3534.93163232037
3515.287896302608
3499.591357765004
3487.0272294027773
3476.9543372510593
3468.8668897211383
3462.3648942587733
3457.1313015587466
3452.9141863953073
3449.512939742377
3446.76746112386
3444.549719731114
3442.7571420792137
3441.307443584241
3440.134484417741
3439.185041217127
3438.416245088883
3437.79354188163
3437.2890311401566
3436.880176353953
3436.548775680453
3436.280105611912
3436.062254744448
3435.8855845839325
3435.7422890810185
3435.6260492986644
3435.5317481066313
3435.455233979358
3435.3931488163753
3435.3427622565205
3435.30186922